In [2]:
import pandas as pd
import numpy as np
import random
import os

In [29]:
accident_data_with_vehicle_data_further_cleaned_w_stations_n_non_accidents = pd.read_csv(
    "../../data/accident_data_with_vehicle_data_further_cleaned_w_stations_n_non_accidents.csv")
df = accident_data_with_vehicle_data_further_cleaned_w_stations_n_non_accidents

__dir_weather_data = "./weather_data_gather/DATASETS/"
df_weather_avg = pd.read_csv(__dir_weather_data + "avg.csv")

In [30]:
df.shape

(1369274, 21)

In [31]:
df.columns = map(str.lower, df.columns)
df.columns

Index(['index', 'longitude', 'latitude', 'accident_severity', 'day_of_week',
       'road_type', 'speed_limit', 'road_surface_conditions',
       'urban_or_rural_area', 'year', 'hour', 'day_of_year', 'cluster_1',
       'vehicle_type', 'sex_of_driver', 'age_of_driver',
       'engine_capacity_(cc)', 'age_of_vehicle', 'driver_home_area_type',
       'station id', 'accident'],
      dtype='object')

In [32]:
df.sample(6)

,index,longitude,latitude,accident_severity,day_of_week,road_type,speed_limit,road_surface_conditions,urban_or_rural_area,year,...,day_of_year,cluster_1,vehicle_type,sex_of_driver,age_of_driver,engine_capacity_(cc),age_of_vehicle,driver_home_area_type,station id,accident
1085716,201301LX513371,-0.105236,51.445922,3.0,2.0,Single carriageway,30.0,Wet/Damp,1.0,2013.0,...,322.0,32430.0,9.0,2.0,44.0,1796.0,5.0,1.0,greater-london_london-st-jamess-park,1
1356696,201130D0000621,-1.476027,52.854615,3.0,6.0,Single carriageway,40.0,Wet/Damp,2.0,2011.0,...,14.0,52007.0,9.0,1.0,18.0,-1.0,-1.0,1.0,derbyshire_morley,1
1097077,201201YE803902,-0.060680,51.614167,0.0,5.0,Dual carriageway,15.0,Frost/Ice,2.0,2005.0,...,323.0,43160.0,9.0,1.0,22.0,119.0,10.0,3.0,greater-london_london-weather-centre,0
437780,200604DF060992,-2.681132,53.766208,0.0,7.0,Single carriageway,40.0,Frost/Ice,2.0,2009.0,...,349.0,6628.0,98.0,1.0,18.0,24.0,9.0,3.0,lancashire_preston-moor-park,0
1097337,201201TB001951,0.121106,51.494833,0.0,6.0,Single carriageway,50.0,Wet/Damp,1.0,2005.0,...,331.0,32872.0,4.0,2.0,44.0,493.0,7.0,-1.0,kent_eynsford,0
1197545,2009400C016741,-0.473507,51.892989,3.0,1.0,Dual carriageway,40.0,Dry,1.0,2009.0,...,165.0,49057.0,19.0,1.0,32.0,1700.0,11.0,1.0,hertfordshire_rothamsted,1


In [33]:
df_weather_avg.sample(6)

,year,day_of_year,hour,wind_direction,wind_speed,air_temperature,dewpoint,wetb_temp,rltv_hum,ground_state_id,cld_ttl_amt_id
25139,2007,318,11,80.877028,2.053437,8.242439,7.931884,8.110062,98.453401,9.982148,4.340665
17809,2007,13,1,230.655645,10.507852,7.305868,4.089109,5.889868,80.429926,9.999894,2.763490
15044,2006,262,20,153.646542,9.756574,12.928198,9.388091,11.165899,79.750257,9.759354,3.295303
28871,2009,108,23,144.094355,9.590362,6.190054,2.552195,4.682235,78.083992,5.920616,4.011524
65460,2013,172,12,293.989088,10.356318,12.991181,8.955913,10.967042,76.863205,6.419978,7.329999
23901,2007,266,21,177.893689,7.427332,11.554596,10.151995,10.852765,91.437658,13.476089,4.122554


In [34]:
# merge average weather data
df = df.merge(df_weather_avg, how='left')

In [35]:
# fine tune with extact weater data using nearest stations 

df_new = df.head(0)

weather_data = os.listdir(__dir_weather_data)
for i, csv in  enumerate(weather_data): 
  df_weather = pd.read_csv(__dir_weather_data + csv)
  station_id = csv.split('.')[0]
  df_weather['station id'] = station_id
  
  df_locations = df.loc[df['station id'] == station_id]
  
  df_locations = df_locations.merge(df_weather, 'left', ['year', 'hour','day_of_year', 'station id'],  suffixes=('*x', '*y')).groupby(lambda x: x.split('*')[0], axis=1).last()
 
  df_new = df_new.append([df_locations],ignore_index=True)
  print('weather:',str(i), '\tstation: ',station_id)

df = df_new


weather: 0 	station:  norfolk_hunstanton-no-2
weather: 1 	station:  aberdeenshire_aberdeen-mannofield-resr
weather: 2 	station:  aberdeenshire_aboyne-no-2
weather: 3 	station:  aberdeenshire_balmoral
weather: 4 	station:  aberdeenshire_braemar-no-2
weather: 5 	station:  aberdeenshire_braemar
weather: 6 	station:  aberdeenshire_cairnorrie-drakemyre-croft
weather: 7 	station:  aberdeenshire_cairnwell
weather: 8 	station:  aberdeenshire_craibstone
weather: 9 	station:  aberdeenshire_dyce
weather: 10 	station:  aberdeenshire_forehill
weather: 11 	station:  aberdeenshire_fyvie-castle
weather: 12 	station:  aberdeenshire_leith-hall-no-2
weather: 13 	station:  aberdeenshire_mintlaw-broombank
weather: 14 	station:  aberdeenshire_oyne
weather: 15 	station:  west-yorkshire_ryhill
weather: 16 	station:  western-isles_barra-w-i
weather: 17 	station:  western-isles_harris-quidnish
weather: 18 	station:  western-isles_skye-lusa
weather: 19 	station:  western-isles_south-uist-range
weather: 20 	stati

In [36]:
df.to_csv("../../data/accident_data_with_vehicle_data_further_cleaned_w_stations_n_non_accidents_weather_merged.csv", index=False)